# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

    *當類別特徵與目標明顯相關時，該考慮採用均值編碼
    *均值編碼的意義在於當一個特徵有明顯意義，但是類別數量特別多的情況使用
    *但最麻煩的點在於極度容易OverFitting， 所以需要不同的平滑化方式

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '../Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

# 對照組 : 標籤編碼 + 邏輯斯迴歸

In [4]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
x = pd.DataFrame()
for i in df.columns:
    x[i] = LabelEncoder().fit_transform(df[i])
x.head()

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [5]:
x_train = x[:train_num]
x_train.shape

(891, 5)

In [6]:
score = cross_val_score(LogisticRegression(), x_train, train_Y, cv=5).mean()
print(f'cross validation score: {score}')

cross validation score: 0.780004837244799


# 均值編碼 + 邏輯斯迴歸

In [63]:
data = pd.concat([df[:train_num], train_Y], axis=1)
for i in object_features:
    group2survived = dict(zip(data.groupby([i])['Survived'].mean().index, data.groupby([i])['Survived'].mean()))
    data[i] = data[i].map(lambda x : group2survived[x])
data.head()

,Name,Sex,Ticket,Cabin,Embarked,Survived
0,0,0.188908,0.0,0.299854,0.336957,0
1,1,0.742038,1.0,1.000000,0.553571,1
2,1,0.742038,1.0,0.299854,0.336957,1
3,1,0.742038,0.5,0.500000,0.336957,1
4,0,0.188908,0.0,0.299854,0.336957,0


In [66]:
x = data[object_features]
y = data['Survived']

In [70]:
start = time.time()
score = cross_val_score(LogisticRegression(), x, y, cv=5).mean()
print(f'cross validation score: {score}')
print(f'time: {time.time() - start} sec')

cross validation score: 1.0
time: 0.04797101020812988 sec


# Method2

In [69]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

score : 1.0
time : 0.028978824615478516 sec


# 結論 

    均值編碼結果較好，但這個結果是無意義的
    因為像Name, Ticket等特徵基本上大部分人都不相同，如果依照生存率進行編碼則會造成Name及Ticket與Survived高度相關
    在進行預測時模型很可能會給特定feature很高的權重，最後造成的結果很可能是Name =1 -> Survived =1